In [1]:
import os
import numpy as np
import pandas as pd

In [19]:
import nltk
from nltk.stem.porter import PorterStemmer

In [3]:
os.listdir('/kaggle/input/spotify-million-song-dataset/spotify_millsongdata.csv')

['spotify_millsongdata.csv']

In [38]:
data=pd.read_csv('/kaggle/input/spotify-million-song-dataset/spotify_millsongdata.csv')

In [39]:
data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [40]:
data.describe()

,artist,song,link,text
count,57650,57650,57650,57650
unique,643,44824,57650,57494
top,Donna Summer,Have Yourself A Merry Little Christmas,/a/abba/ahes+my+kind+of+girl_20598417.html,I just came back from a lovely trip along the ...
freq,191,35,1,6


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


Since the column `'link'` is not useful for us, we will drop this column.

In [42]:
data=data.sample(2500).drop('link',axis=1).reset_index(drop=True)

In [43]:
data.head()

,artist,song,text
0,Usher,There Goes My Baby,"Yeah, \r\nRight, \r\nUsher baby, OK \r\nYea..."
1,Hank Snow,Come The Morning,I sit upon the bed in my rented room and watch...
2,Xandria,Answer,We fought the truth \r\nAs long as we lived o...
3,Cheap Trick,She's Got Motion,Met the girl said to me \r\nWhat is it you wa...
4,HIM,Motor,"Motor \r\n \r\nStill can't believe it, can't..."


In [44]:
data['text'][0]

"Yeah,  \r\nRight,  \r\nUsher baby, OK  \r\nYeah man, right  \r\n  \r\nThere goes my baby  \r\n(ooh girl look at you)  \r\nYou don't know how, good, it,  \r\nFeels to call you my girl  \r\nThere goes my baby  \r\nLoving everything you do  \r\nOoh girl look at you  \r\n  \r\nBet you ain't know that I be checking you out  \r\nWhen you be putting your heels on  \r\nI swear your body's so perfect baby  \r\nHow you work it baby yea  \r\nI love the way that you be poking it out  \r\nGirl give me something to feel on  \r\nSo please believe we gone be twerking it out  \r\nBy the end of the night baby  \r\n  \r\nI've been waiting all day to wrap my hands  \r\nAround your waist and kiss your face  \r\nWouldn't trade this feeling for nothing  \r\nNot even for a minute  \r\nAnd I'll sit here long as it takes  \r\nTo get you all alone  \r\nBut as soon as you come walking my way  \r\nYou gon' hear me say  \r\n  \r\nThere goes my baby (there she goes, there she goes, there she goes)  \r\n(ooh girl lo

In [45]:
data=data.sample(2000)

## Preprocessing Data

In [46]:
data['text']=data['text'].str.lower().replace(r'a-zA-Z0-9',' ').replace(r'\n',' ',regex=True)

In [47]:
stemmer=PorterStemmer()

In [48]:
def token(text):
    token=nltk.word_tokenize(text)
    l=[stemmer.stem(word) for word in token]
    return " ".join(l)

In [49]:
token("Machine learning is very interesting")

'machin learn is veri interest'

In [50]:
data['text'].apply(lambda x:token(x))

825     alon with the phone again the deepen light see...
915     jojo wa a man who thought he wa a woman , but ...
81      ai n't found a way to kill me yet eye burn wit...
1894    i ai n't blue , babi just a littl bit lonesom ...
2268    do noth till you hear from me pay no attent to...
                              ...                        
164     thank you , babi for the year you 've given me...
1879    [ tiffani ] amu geotdo mitgo sipji anhatji ( o...
1707    all my life , i worship her her golden voic , ...
2311    train arriv sixteen coach long train arriv six...
166     night- and the world is fast asleep i will hol...
Name: text, Length: 2000, dtype: object

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [52]:
tfid=TfidfVectorizer(analyzer='word',stop_words='english')

In [53]:
m=tfid.fit_transform(data['text'])

In [54]:
similarity=cosine_similarity(m)
similarity

array([[1.        , 0.        , 0.0115765 , ..., 0.0238211 , 0.02754685,
        0.01524154],
       [0.        , 1.        , 0.01748625, ..., 0.02264494, 0.01526518,
        0.02075641],
       [0.0115765 , 0.01748625, 1.        , ..., 0.02765378, 0.00677771,
        0.0057893 ],
       ...,
       [0.0238211 , 0.02264494, 0.02765378, ..., 1.        , 0.04560032,
        0.01274732],
       [0.02754685, 0.01526518, 0.00677771, ..., 0.04560032, 1.        ,
        0.01493998],
       [0.01524154, 0.02075641, 0.0057893 , ..., 0.01274732, 0.01493998,
        1.        ]])

In [55]:
data['song'].head()

825                        Long Distance
915                             Get Back
81                               Rooster
1894                        I Ain't Blue
2268    Do Nothing Till You Hear From Me
Name: song, dtype: object

## Creating a Function for Recommending Songs

In [62]:
def recommend_songs(song_name):
    index=data[data['song']==song_name].index[0]
    dist=sorted(list(enumerate(similarity[index])),reverse=True,key=lambda x:x[1])
    song=[]
    for song_id in dist[1:6]:
        song.append(data.iloc[song_id[0]].song)
    return song

In [63]:
recommend_songs('Long Distance')

['When Two Worlds Collide',
 "Something's Gotta Give",
 'Snowflake',
 "What I'll Never Find",
 'In The Night']